In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import torch
import torchaudio as ta
import torchaudio.functional as taf
import torchaudio.transforms as tat
from torchvision import transforms

print(torch.__version__)        
print(ta.__version__)

import matplotlib
import matplotlib.pyplot as plt
from IPython.display import Audio, display

import pandas as pd
import os
import pprint

from typing import *
import itertools
from collections import Counter

import numpy as np
from datetime import datetime
from lark.config import Config
from lark.learner import Learner
from lark.ops import Sig2Spec, MixedSig2Spec

1.8.1
0.8.0a0+e4e171a


In [3]:
cfg = Config(
#     n_mels = 128,
#     f_min = 20,
#     f_max = 16000,
#     n_fft = 2048,
#     hop_length = 512,
    
    noise_nsr_dbs = [35, 30, 20, 10],
    site='SSW', 
    use_neptune=True, 
    n_epochs=15, 
    bs=32, 
    lr=1e-4, 
    model='densenet',
    scheduler='torch.optim.lr_scheduler.OneCycleLR'
)

In [4]:
cfg.as_dict()

{'site': 'SSW',
 'data_dir': 'data/birdclef-2021',
 'checkpoint_dir': 'checkpoints',
 'bs': 32,
 'n_workers': 12,
 'training_dataset_size': 10752,
 'duration': 5,
 'use_noise': True,
 'noise_nsr_dbs': [35, 30, 20, 10],
 'noise_dir': 'data/noise/BirdVox-DCASE-20k/wav-32k',
 'use_overlays': True,
 'max_overlays': 5,
 'overlay_weights': [0.71986223,
  0.21010333,
  0.06314581,
  0.00574053,
  0.00114811],
 'overlay_snr_dbs': [20, 10, 3],
 'use_neptune': True,
 'sr': 32000,
 'n_frames': 160000,
 'n_fft': 512,
 'window_length': 512,
 'n_mels': 64,
 'hop_length': 312,
 'f_min': 150,
 'f_max': 15000,
 'lr': 0.0001,
 'n_epochs': 15,
 'model': 'densenet',
 'optimizer': 'torch.optim.Adam',
 'loss_fn': 'torch.nn.BCEWithLogitsLoss',
 'scheduler': 'torch.optim.lr_scheduler.OneCycleLR',
 'labels': ['amecro',
  'amegfi',
  'amerob',
  'balori',
  'belkin1',
  'bkcchi',
  'blujay',
  'cangoo',
  'chswar',
  'comgra',
  'comyel',
  'dowwoo',
  'eastow',
  'eawpew',
  'gockin',
  'grycat',
  'haiwoo',
 

In [6]:
main_model = torch.hub.load('pytorch/vision:v0.9.0', 'densenet121', pretrained=True)
main_model

Using cache found in /home/koen/.cache/torch/hub/pytorch_vision_v0.9.0


DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

In [7]:
prep = MixedSig2Spec(cfg)

posp = torch.nn.Linear(in_features=1024, 
                       out_features=len(cfg.labels), bias=True)

# for param in posp.parameters():
#     param.requires_grad = True

main_model.classifier = posp

model = torch.nn.Sequential(prep, main_model)

model = model.cuda()


In [9]:
lrn = Learner("densenet", cfg, model)

In [10]:
lrn.learn()

psutil is not installed. You will not be able to abort this experiment from the UI.
psutil is not installed. Hardware metrics will not be collected.


https://ui.neptune.ai/botkop/lark/e/LAR-63


  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/336 [00:00<?, ?it/s]

  0%|          | 0/38 [00:00<?, ?it/s]

...

  0%|          | 0/38 [00:00<?, ?it/s]

2021-05-03 17:04:52 epoch:  14 train loss: 0.119154 train f1: 0.523736 valid loss: 0.090301 valid f1: 0.305694


  0%|          | 0/336 [00:00<?, ?it/s]

  0%|          | 0/38 [00:00<?, ?it/s]

2021-05-03 17:06:18 epoch:  15 train loss: 0.118455 train f1: 0.532611 valid loss: 0.092844 valid f1: 0.293820


In [11]:
lrn.scheduler.state_dict()

{'total_steps': 5040,
 '_schedule_phases': [{'end_step': 1511.0,
   'start_lr': 'initial_lr',
   'end_lr': 'max_lr',
   'start_momentum': 'max_momentum',
   'end_momentum': 'base_momentum'},
  {'end_step': 5039,
   'start_lr': 'max_lr',
   'end_lr': 'min_lr',
   'start_momentum': 'base_momentum',
   'end_momentum': 'max_momentum'}],
 'anneal_func': <bound method OneCycleLR._annealing_cos of <torch.optim.lr_scheduler.OneCycleLR object at 0x7ff524864280>>,
 'cycle_momentum': True,
 'use_beta1': True,
 'base_lrs': [4e-05],
 'last_epoch': 5040,
 '_step_count': 5041,
 'verbose': False,
 '_get_lr_called_within_step': False,
 '_last_lr': [4.198235228265301e-09]}

In [12]:
# lrn.name = 'resnest50-mixed-spec-half-frozen-20210502-144336-latest'

In [13]:
# lrn.save_checkpoint(50, 0.097231, 0.234157)

In [14]:
lrn.evaluate()

  0%|          | 0/38 [00:00<?, ?it/s]

,thresh,tp,tn,fp,fn,f1
0,0.0,770,0,32830,0,0.044807
1,0.1,416,31162,1668,354,0.291521
2,0.2,293,32434,396,477,0.401645
3,0.3,235,32669,161,535,0.403087
4,0.4,188,32739,91,582,0.358437
5,0.5,160,32768,62,610,0.322581
6,0.6,128,32793,37,642,0.273797
7,0.7,112,32806,24,658,0.247241
8,0.8,85,32812,18,685,0.194731
9,0.9,54,32823,7,716,0.129964


In [15]:
lrn.load_checkpoint()

{'epoch': 13,
 'valid_loss': 0.08691110412933324,
 'valid_score': 0.37181997299194336}

In [16]:
lrn.evaluate()

  0%|          | 0/38 [00:00<?, ?it/s]

,thresh,tp,tn,fp,fn,f1
0,0.0,770,0,32830,0,0.044807
1,0.1,443,31180,1650,327,0.309466
2,0.2,332,32424,406,438,0.440318
3,0.3,273,32657,173,497,0.449013
4,0.4,221,32723,107,549,0.402550
5,0.5,190,32768,62,580,0.371820
6,0.6,153,32785,45,617,0.316116
7,0.7,131,32800,30,639,0.281418
8,0.8,101,32812,18,669,0.227222
9,0.9,65,32823,7,705,0.154394
